## Importing minute data

In [1]:
import pandas as pd
import datetime
import numpy as np
min_data = pd.read_csv('../Data/nifty/min/min_consolidated.csv')
holidays = pd.read_csv('../Data/nifty/holidays.csv')


In [2]:
min_data['datetime']=min_data['date']+" "+min_data['time']
min_data['datetime'] =  pd.to_datetime(min_data['datetime'], infer_datetime_format=True)
cols = ['datetime','open','high','low','close']
min_data = min_data[cols]
min_data = min_data.set_index("datetime")
# min_data.index = min_data.index - pd.Timedelta(minutes=1)

holidays['date'] = holidays['date'].astype('datetime64[ns]')

### 1 Minute
### Filter with 09:55<time<3:30 before 31-12-2009, and values not null after 3:30 

In [3]:
# group in 1-minute chunks. 
# res = min_data.resample('5Min', on='date_time_from_epoch').agg(d).reset_index()
t = min_data.groupby(pd.Grouper(freq='1Min')).agg({"open": "first", 
                                             "high": "max", 
                                             "low": "min", 
                                             "close": "last"})
# t.to_clipboard(excel = False, sep = ',')
t

,open,high,low,close
datetime,,,,
2007-01-02 09:55:00,3978.2,3978.40,3966.40,3978.40
2007-01-02 09:56:00,3978.9,3980.70,3978.90,3980.40
2007-01-02 09:57:00,3980.3,3981.30,3979.15,3979.15
2007-01-02 09:58:00,3979.8,3980.15,3979.25,3980.15
2007-01-02 09:59:00,3979.5,3979.60,3979.30,3979.30
...,...,...,...,...
2021-12-31 15:26:00,17357.9,17358.50,17353.80,17356.30
2021-12-31 15:27:00,17356.1,17359.70,17355.60,17359.30
2021-12-31 15:28:00,17358.8,17360.50,17356.20,17359.80


In [5]:
## data between regular market hours
t_before = t[t.index.date < datetime.date(2011,1,4)]
t_after = t[t.index.date > datetime.date(2011,1,3)]

t_before = t_before[t_before.index.time < datetime.time(15,31)]
t_before = t_before[t_before.index.time > datetime.time(9,54)]
# 2010-10-18
t_after = t_after[t_after.index.time < datetime.time(15,31)]
t_after = t_after[t_after.index.time > datetime.time(9,0)]

In [6]:
## combined market hours data
combined_1min = pd.concat([t_before, t_after])
combined_1min = combined_1min.sort_index(ascending=True)

## Data out of market hours
others = t[~t.isin(combined_1min)].dropna()
combined_1min = pd.concat([combined_1min, others], sort=False)
combined_1min = combined_1min.sort_index(ascending=True)
# combined_1min = combined_1min.drop_duplicates()
# combined_1min = combined_1min.interpolate(method='linear')

In [7]:
## Data in weekends and weekdays
combined_1min_weekdays = combined_1min[combined_1min.index.dayofweek < 5]
combined_1min_weekends = combined_1min[combined_1min.index.dayofweek > 4]
## Data in weekends and not null  --> currently removing all null data. But need to remove na only if no values available on non-traded days
    ## logic is to find unique days -> remove all null data on other days than in unique days.
    #weekends_unique = pd.to_datetime(pd.DataFrame(combined_1min_weekends.index.floor('D').unique())['datetime']).dt.date.unique().tolist()
combined_1min_weekends = combined_1min_weekends[combined_1min_weekends['open'].notna()]
## Data in weekdays and not null in weekends
combined_1min = pd.concat([combined_1min_weekdays,combined_1min_weekends])
combined_1min = combined_1min.sort_index(ascending=True)


In [8]:
## Data in holidays
holidays_list = pd.to_datetime(holidays['date']).dt.date.unique().tolist()
combined_1min_holidays = combined_1min[combined_1min.index.floor('D').isin(holidays_list)]
## Data in holidays and not null  --> currently removing all null data. But need to remove na only if no values available on non-traded days
    ## logic is to find unique days -> remove all null data on other days than in unique days.
combined_1min_holidays = combined_1min_holidays[combined_1min_holidays['open'].notna()]
## combining Data in holidays with original df
combined_1min_nonholidays = combined_1min[~combined_1min.index.floor('D').isin(holidays_list)]
combined_1min = pd.concat([combined_1min_nonholidays,combined_1min_holidays])
combined_1min = combined_1min.sort_index(ascending=True)



### Nifty Conclusion (After 1 minute adjustment)
Till 2009-10-22 (including)-> Trading from 09:55

2009-10-23(inc) to 2009-12-31(inc) -> Trading from 09:56

2010-01-04 -> Trading from 09:31

From 2010-01-05 (incl) -> Trading from 09:01

2010-10-18 -> preopen at 09:08 and trading started from 09:16

In [ ]:
def nifty_time_adjustment(time_index):
    if time_index < datetime.datetime(2009,10,22,23,59,59):
        return time_index
    elif datetime.datetime(2009,10,23,00,1,1) < time_index < datetime.datetime(2009,12,31,23,59,59):
        return time_index - pd.Timedelta(minutes=1)
    elif time_index.date()==datetime.date(2010,1,4):
        return time_index - pd.Timedelta(minutes=1)
    elif datetime.datetime(2010,1,5,00,1,1) < time_index < datetime.datetime(2010,10,15,23,59,59):
        return time_index - pd.Timedelta(minutes=1)
    elif time_index > datetime.datetime(2010,10,18,00,1,1):
        return time_index - pd.Timedelta(minutes=1)


In [9]:
# Making time adjustments as per the inferences/conclusion mentioned above.
combined_1min['date']= combined_1min.index
combined_1min['date'] = combined_1min['date'].apply(lambda x:nifty_time_adjustment(x))
combined_1min.set_index('date', inplace=True)

In [34]:
## Remove data from 9:00 to 9:14 (pre-opening) from 2010-10-17
combined_1min = combined_1min.drop(combined_1min[(datetime.time(8,59,0)<combined_1min.index.time) & (combined_1min.index.time<datetime.time(9,15,0)) & (combined_1min.index.date>datetime.date(2010,10,17))].index)
## Remove data with NaN at 15:30
combined_1min = combined_1min.drop(combined_1min[(combined_1min.index.time==datetime.time(15,30,00)) & (combined_1min.open.isnull())].index)

In [35]:
combined_1min.to_csv("combined_1mins.csv",index=True)

### 5 Minutes Conversion
### Filter with 09:55<time<3:30 before 31-12-2009, and values not null after 3:30 

In [ ]:
# group in 5-minute chunks. 
# res = min_data.resample('5Min', on='date_time_from_epoch').agg(d).reset_index()
t = min_data.groupby(pd.Grouper(freq='5Min')).agg({"open": "first", 
                                             "high": "max", 
                                             "low": "min", 
                                             "close": "last"})
# t.to_clipboard(excel = False, sep = ',')
t

In [44]:
## data between regular market hours
t_before = t[t.index.date < datetime.date(2011,1,4)]
t_after = t[t.index.date > datetime.date(2011,1,3)]

t_before = t_before[t_before.index.time < datetime.time(15,31)]
t_before = t_before[t_before.index.time > datetime.time(9,54)]

t_after = t_after[t_after.index.time < datetime.time(15,31)]
t_after = t_after[t_after.index.time > datetime.time(9,0)]

In [48]:
combined_5min = pd.concat([t_before, t_after])
combined_5min = combined_5min.sort_index(ascending=True)

others = t[~t.isin(combined_5min)].dropna()
combined_5min = pd.concat([combined_5min, others], sort=False)
combined_5min = combined_5min.sort_index(ascending=True)
combined_5min = combined_5min.drop_duplicates()
combined_5min = combined_5min.interpolate(method='linear')

In [49]:
combined_5min.to_csv("combined_5mins.csv",index=True)

## 5min from 1 minute data

In [47]:
# group in 1-minute chunks. 
# res = min_data.resample('5Min', on='date_time_from_epoch').agg(d).reset_index()
t_5min = combined_1min.groupby(pd.Grouper(freq='5Min')).agg({"open": "first", 
                                             "high": "max", 
                                             "low": "min", 
                                             "close": "last"})
# t.to_clipboard(excel = False, sep = ',')
t_5min

,open,high,low,close
datetime,,,,
2007-01-02 09:50:00,3978.20,3978.40,3966.40,3978.40
2007-01-02 09:55:00,3978.90,3981.30,3976.95,3977.25
2007-01-02 10:00:00,3978.60,3980.10,3972.75,3973.20
2007-01-02 10:05:00,3967.00,3975.40,3965.55,3973.40
2007-01-02 10:10:00,3973.65,3973.65,3968.30,3971.05
...,...,...,...,...
2021-12-31 15:05:00,17354.30,17357.40,17350.00,17356.70
2021-12-31 15:10:00,17356.40,17357.20,17349.70,17351.40
2021-12-31 15:15:00,17351.40,17359.30,17350.10,17358.50
